# Day 11, possibly dynamic programming?

This is a challenge that, to me, sounds like we need to use dynamic programming. For a large problem set, you'd only want to keep a running total. Since we want the top-left grid coordinate, you'd start at `(max(x), max(y))` and work your way backwards, taking advantage of the calculations already done for the `x + 1, y`, `x + 2, y`, `x, y + 1`, ..., `x + 2, y + 2` positions. For a 300 x 300 grid that would mean you only need to keep the last 600 calculation results in memory and let you use `max()` on the running calculation.

However, for a 300x300 grid it is simpler to just vectorise the hell out of the grid, using `numpy`.

The formula for any given grid coordinate power value is

$$\lfloor\frac{((x + 10)y + serial) \times (x + 10)}{100}\rfloor \bmod 10 - 5$$

but I must note that subtracting 5 at the end doesn't actually matter to the outcome. Either the cell score falls in the range $[0, 10)$ or $[-5, 5)$, with the 3 x 3 grid score in the range $[0, 81]$ or $[-45, 36]$. Not that `numpy` much cares.

Summing the sliding 3x3 windows is a little more interesting here. There are 298 x 298 complete 3 x 3 sub-grids that need to be considered here (from `((1, 1) ... (3, 3))` all the way to `((298, 298) ... (300, 300))`), and we need to create sums for all those sub windows. I'm using the [`numpy.lib.stride_tricks.as_strided()` function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.lib.stride_tricks.as_strided.html) here to step over the whole matrix in those 3x3 windows, so we can sum them all and produce a new `(298 x 298)` matrix of sums at coordinates that match the top-level corner of each sub-matrix.

In [1]:
from itertools import product
import numpy as np

# These values never change, so can be made globals
GRIDSIZE = 300
XX, YY = np.meshgrid(np.arange(1, GRIDSIZE + 1), np.arange(1, GRIDSIZE + 1))
RACK_ID = XX + 10
# Sliding window sizes, per side
WINDOW_SIZE = 3
WINDOW_COUNT = (GRIDSIZE - WINDOW_SIZE + 1)

def grid_powers(serial):
    # calculate power levels
    return (RACK_ID * YY + serial) * RACK_ID // 100 % 10 - 5
    
def max_grid(serial):    
    # sum levels for 3 x 3 subgrids; substitute edges for zeros
    power_levels = grid_powers(serial)
    
    # we want to sum every subwindow, so it is time to start striding
    # we need to produce a (WINDOW_COUNT, WINDOW_COUNT, WINDOW_SIZE, WINDOW_SIZE)
    # matrix that then is summed on the last 2 axes.
    summed = np.lib.stride_tricks.as_strided(
        power_levels,
        # output shape, 2d grid of 2d windows
        (WINDOW_COUNT, WINDOW_COUNT, WINDOW_SIZE, WINDOW_SIZE),
        # per shape axis, the stride across power_levels matches up to the
        # same axes.
        power_levels.strides + power_levels.strides
    ).sum(axis=(2, 3))
    
    # produce the (x, y) coordinates for the largest 3x3 grid top-left coordinate
    # argmax() flattens the array and gives us an index based on that, so we need
    # numpy.unravel to give back the original y, x coordinates.
    y, x = np.unravel_index(summed.argmax(), summed.shape)
    # Translate from zero to one-based indexing
    return x + 1, y + 1

In [2]:
power_tests = {
    # serial, x, y: power level
    (8, 3, 5): 4,
    (57, 122, 79): -5,
    (39, 217, 196): 0,
    (71, 101, 153): 4,
}

for (serial, x, y), expected in power_tests.items():
    # indexing a [y, x] arranged matrix with 0-based offsets
    assert grid_powers(serial)[y - 1, x - 1] == expected

max_tests = {
    18: (33, 45),
    42: (21, 61),
}

for serial, expected in max_tests.items():
    assert max_grid(serial) == expected

In [3]:
import aocd

data = aocd.get_data(day=11, year=2018)
serial = int(data)

In [4]:
x, y = max_grid(serial)
print(f'Part 1: {x},{y}')

Part 1: 44,37
